In [14]:
import glob
list_of_files = glob.glob('datasets/RESC/pseudo_label_2/*')

In [15]:
from PIL import Image
import numpy as np
def _fast_hist(label_true, label_pred, n_class):
    mask = (label_true >= 0) & (label_true < n_class)
    hist = np.bincount(
        n_class * label_true[mask].astype(int) + label_pred[mask],
        minlength=n_class ** 2,
    ).reshape(n_class, n_class)
    return hist
def scores(label_trues, label_preds, n_class):
    accumulate_hist = np.zeros((n_class, n_class))
    for lt, lp in zip(label_trues, label_preds):
        accumulate_hist += _fast_hist(lt.flatten(), lp.flatten(), n_class)
    # TP / all
    acc = np.diag(accumulate_hist).sum() / accumulate_hist.sum()
    # TP / (TP + FN)
    acc_cls = np.diag(accumulate_hist) / accumulate_hist.sum(axis=1)
    acc_cls = np.nanmean(acc_cls)

    intersection = np.diag(accumulate_hist)
    union = accumulate_hist.sum(axis=1) + accumulate_hist.sum(axis=0) - intersection
    iu = intersection / union
    freq = accumulate_hist.sum(axis=1) / accumulate_hist.sum()
    fwavacc = (freq[freq > 0] * iu[freq > 0]).sum()
    valid = accumulate_hist.sum(axis=1) > 0  # added
    mean_iu = np.nanmean(iu[valid])
    cls_iu = dict(zip(range(n_class), iu))
    # import pdb; pdb.set_trace()

    return {
        "Pixel Accuracy": acc,
        "Mean Accuracy": acc_cls,
        "Frequency Weighted IoU": fwavacc,
        "Mean IoU": mean_iu,
        "Class IoU": cls_iu,
    }
def convert_resc_labels(img):
    # 0 background,
    # 1 lesion(need turn to background),  1 -> 0
    # 0.74 SRF(need turn to 1),           0.74 -> 1
    # 0.51 PED(need turn to 2)            0.51 -> 2
    # back: 0, ped: 128, srf: 191, retinal: 255
    img[img == 255] = 0
    img[img == 191] = 1
    img[img == 128] = 2
    return img
pred_list = []
gt_list = []
for each_file in list_of_files:
    name = each_file.split('/')[-1].split('.')[0]
    predict = np.array(Image.open(each_file))
    gt_dir = 'datasets/RESC/train/label_images/' + name + '.bmp'
    gt = np.array(Image.open(gt_dir))
    gt = convert_resc_labels(gt)
    pred_list.append(predict)
    gt_list.append(gt)

score = scores(gt_list, pred_list, n_class=3)

In [16]:
score

{'Pixel Accuracy': 0.9788560660726385,
 'Mean Accuracy': 0.6500623565700147,
 'Frequency Weighted IoU': 0.9629158298008386,
 'Mean IoU': 0.5387106503954331,
 'Class IoU': {0: 0.9788435252417673,
  1: 0.34947444311318937,
  2: 0.28781398283134274}}